# LDA+DNN

## 1.导入工具包

In [1]:
!pip install gensim jieba

In [2]:
# import os
import csv
import time
import datetime
import random
import json
from collections import Counter
from math import sqrt
import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed,Input,Conv2D,MaxPool2D,concatenate,Flatten,Dense,Dropout,Embedding,SimpleRNN,Reshape,GRU,LSTM
from tensorflow.keras import Sequential,optimizers,losses
from tensorflow.keras.models import Model,model_from_yaml
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from bs4 import BeautifulSoup
import logging
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import multiprocessing
import yaml
import jieba
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## 2.设置参数

In [3]:
vocab_dim = 60
maxlen = 50
n_iterations = 1  # ideally more..
n_exposures = 10
window_size = 7
batch_size = 64
n_epoch = 10
input_length = 60
cpu_count = multiprocessing.cpu_count()

## 3.读取数据集

In [4]:
!pwd

/home/jovyan/ai-lab/Day10-NLP


In [5]:
neg=pd.read_excel('../../data/day10-nlp-data/film_review/neg.xlsx',header = None)
pos=pd.read_excel('../../data/day10-nlp-data/film_review/pos.xlsx',header = None)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/day10-nlp-data/film_review/neg.xlsx'

## 4. 查看数据集

### 消极评论

In [ ]:
neg.head()

### 积极评论

In [ ]:
pos.head()

## 5.数据连接

In [ ]:
comment=np.concatenate((pos[0], neg[0]))
y = np.concatenate((np.ones(len(pos),dtype=int), np.zeros(len(neg),dtype=int)))
comment_new = comment.astype(str).tolist()

### 6.分词预处理

In [ ]:
def get_stopword_list():
    """
    建立停用词表
    """
    stop_word_path = '../../data/day10-nlp-data/chineseStopWords.txt'
    stopword_list = [sw.replace('\n', '').replace(' ', '') for sw in open(stop_word_path, encoding='gb18030').readlines()]
    return stopword_list
stopword_list = get_stopword_list()

In [ ]:
stopword_list.append('@')
stopword_list.append(' ')
stopword_list.append('')
stopword_list.append('..........................................................................................')
stopword_list.append('\n')
stopword_list.append('$')

def cut_remove(text):
    """
    分词
    """
    news_list = []
    for str_text in text:
        str_words = ','.join(jieba.cut(str_text)).split(',')
        for word in str_words:
            if word not in stopword_list:
                news_list.append(word)
    return news_list

In [ ]:
comment_new = cut_remove(comment_new)

## 7.LDA主题模型

In [ ]:
"""
1.加载gensim，建立lda模型
"""
dictionary = Dictionary([comment_new])
print(dictionary)
corpus = [dictionary.doc2bow(text) for text in [comment_new]]

"""
2.使用LDA算法求得前5的topic，同时生成的topic在之后也会被使用到来定义文本所属主题  
"""
 
NUM_TOPICS = 5
ldamodel = LdaModel(corpus,num_topics = NUM_TOPICS, id2word=dictionary, passes=15)

"""
3.保存lda模型
"""
ldamodel.save('model/LDA/model5.gensim')


topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

In [ ]:
comment

In [ ]:
def lda2list(lda,topic_n):
    lda_dict = dict(lda)
    lda_list = [0] * topic_n
    for i in range(topic_n):
        lda_list[i] = lda_dict.get(i,0)
    return lda_list

comment = comment.astype(str).tolist()
comment = [','.join(jieba.cut(document)).split(',') for document in comment]


def remove_stopwords(text):
    """
    去停用词
    """
    for index in text[:]:
        if index in stopword_list:
            text.remove(index)
    return text       
comment = [remove_stopwords(document) for document in comment]    

# 句子主题向量化
comment_vec =[]
for comment_content in comment:
    comment_bow = dictionary.doc2bow(comment_content)     
    comment_lda = lda2list(ldamodel[comment_bow],6) 
    comment_vec.append(comment_lda)

In [ ]:
X = np.array(comment_vec)

## 8.加载词向量模型

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
x_train

## 9.创建模型

In [ ]:
"""
1.初始化：
    glorot_uniform均匀分布初始化
    优化函数Adam
"""

init = K.initializers.glorot_uniform(seed=2)
simple_adam = K.optimizers.Adam()

"""
2.定义DNN模型：
    搭建的DNN由输入层、隐藏层、输出层和sigmoid函数组成，其中输入层由训练集的维度决定，隐藏层有一层，
    包括32个神经元，之后就是输出层，由1个神经元组成，最后，sigmoid，用于解决二分类问题而创建。
"""
model = K.models.Sequential()
model.add(K.layers.Dense(units=32, input_dim=x_train.shape[1], kernel_initializer=init, activation='sigmoid'))
model.add(K.layers.Dense(units=1, kernel_initializer=init, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=simple_adam, metrics=['accuracy'])

"""
3.训练模型fit
"""
h = model.fit(X_new_train, y_train, batch_size=100, epochs=5, shuffle=True, verbose=1)

"""
4.检验模型evaluate
"""
eval = model.evaluate(X_test, y_test, verbose=0)
print("Evaluation on test data: loss = %0.6f accuracy = %0.2f%% \n" \
          % (eval[0], eval[1] * 100))